# Variational Quantum Eigensolver (VQE)

Para este módulo, os estudantes devem ter um **ambiente Python funcional** e as **versões mais recentes** dos seguintes pacotes instalados:

*   `qiskit`
*   `qiskit_ibm_runtime`
*   `qiskit-aer`
*   `qiskit.visualization`
*   `numpy`
*   `pylatexenc`

Para configurar e instalar esses pacotes, consulte o guia [Instalar o Qiskit](/docs/guides/install-qiskit). Para executar tarefas em computadores quânticos reais, os estudantes precisarão configurar uma conta no **IBM Cloud**, seguindo as etapas descritas no guia [Configurar sua conta IBM Cloud](/docs/guides/cloud-setup).


Este módulo foi testado e utilizou aproximadamente 8 minutos de tempo de QPU. Esse valor é apenas uma estimativa, e o uso real pode variar.

In [ ]:
# Uncomment and modify this line as needed to install dependencies
#!pip install 'qiskit>=2.1.0' 'qiskit-ibm-runtime>=0.40.1' 'qiskit-aer>=0.17.0' 'numpy' 'pylatexenc'

## Introdução

Desde o desenvolvimento do modelo da mecânica quântica no início do século XX, os cientistas compreenderam que os elétrons não seguem trajetórias fixas ao redor do núcleo de um átomo, mas existem em regiões de probabilidade chamadas orbitais. Esses orbitais correspondem a níveis de energia específicos e discretos que os elétrons podem ocupar. Os elétrons naturalmente permanecem nos níveis de energia mais baixos disponíveis, conhecidos como estado fundamental. No entanto, se um elétron absorver energia suficiente, ele pode saltar para um nível de energia mais alto, entrando em um estado excitado. Esse estado excitado é temporário, e o elétron eventualmente retorna a um nível de energia mais baixo, liberando a energia absorvida, muitas vezes na forma de luz. Esse processo fundamental de absorção e emissão de energia é essencial para compreender como os átomos interagem e formam ligações.

Quando átomos se unem para formar moléculas, seus orbitais atômicos se combinam para formar orbitais moleculares. A disposição e os níveis de energia dos elétrons nesses orbitais moleculares determinam as propriedades da molécula resultante e a força das ligações químicas. Por exemplo, na formação de uma molécula de hidrogênio ($H_2$) a partir de dois átomos individuais de hidrogênio, o elétron de cada átomo ocupa um orbital atômico. À medida que os átomos se aproximam, esses orbitais atômicos se sobrepõem e se combinam para formar novos orbitais moleculares — um de menor energia (um orbital ligante) e outro de maior energia (um orbital antiligante). Os dois elétrons, um de cada átomo de hidrogênio, ocupam preferencialmente o orbital ligante de menor energia, levando à formação de uma ligação covalente estável que mantém a molécula de $H_2$ unida. A diferença de energia entre os átomos separados e a molécula formada, em particular a energia dos elétrons nos orbitais moleculares, determina a estabilidade e as propriedades da ligação.

Nas seções seguintes, exploraremos esse processo de formação molecular, com foco na molécula de $H_2$. Utilizaremos um computador quântico real, combinado com técnicas clássicas de otimização, para encontrar a energia desse processo simples, porém fundamental. Esse experimento fornecerá uma demonstração prática de como a computação quântica pode ser aplicada para resolver problemas de química computacional, oferecendo insights sobre o papel da energia dos elétrons.

## VQE – Um algoritmo quântico variacional para problemas de autovalores

### Técnicas de aproximação em química – princípio variacional e conjunto de bases

As contribuições de Erwin Schrödinger para a mecânica quântica não se limitam à introdução de um novo modelo eletrônico; de forma fundamental, ele estabeleceu a mecânica ondulatória ao desenvolver a famosa equação de Schrödinger dependente do tempo:


$$
i\hbar \frac{d}{dt}|\psi\rangle = \hat{H}|\psi\rangle
$$

Aqui, $\hat{H}$ é o operador Hamiltoniano, que representa a energia total do sistema, e $|\psi\rangle$ é a função de onda que contém todas as informações sobre o estado quântico do sistema. (Nota: $\frac{d}{dt}$ é a derivada total em relação ao tempo, e não incluímos explicitamente aqui o autovalor de energia $E$.)


No entanto, em muitas aplicações práticas — como a determinação dos níveis de energia permitidos de átomos e moléculas — utiliza-se, em vez disso, a equação de Schrödinger independente do tempo (equação de autovalores de energia), que é derivada da forma dependente do tempo ao se assumir um estado estacionário. Um estado estacionário é um estado quântico no qual a densidade de probabilidade de encontrar uma partícula em um determinado ponto do espaço não se altera ao longo do tempo.

$\hat{H}|\psi\rangle = E|\psi\rangle$

Nessa forma, $E$ representa o autovalor de energia correspondente ao estado quântico $|\psi\rangle$. O Hamiltoniano inclui diversas contribuições de energia, como a energia cinética dos elétrons e dos núcleos, as forças atrativas entre elétrons e núcleos e as forças repulsivas entre os próprios elétrons.

Resolver a equação de autovalores de energia nos permite calcular os níveis de energia quantizados de sistemas atômicos e moleculares. No entanto, para moléculas, resolvê-la exatamente é difícil porque a função de onda $\Psi$, que descreve a distribuição espacial dos elétrons, é complexa e de alta dimensionalidade.

Como resultado, os cientistas utilizam técnicas de aproximação para obter soluções práticas e precisas. Neste trabalho, iremos nos concentrar em dois métodos principais:

1. **Princípio variacional**

   Esse método aproxima a função de onda e a ajusta para chegar o mais próximo possível da energia desejada, geralmente a energia do estado fundamental do sistema. A ideia central por trás do princípio variacional é simples:

   * Se supusermos uma função de onda $\Psi_\text{trial}$ (uma “função de teste”), a energia calculada a partir dela será sempre maior ou igual à energia do estado fundamental ($E_0$) do sistema:
     $$
     E_\text{approx} = \frac{\langle \Psi_\text{trial}|\hat{H}|\Psi_\text{trial}\rangle}{\langle \Psi_\text{trial}|\Psi_\text{trial}\rangle} \geq E_0
     $$
   * Ao ajustar os parâmetros $\theta$ na função de teste, $|\Psi_\text{trial}(\theta)\rangle$, podemos obter aproximações cada vez melhores da energia do estado fundamental.
   * A precisão desse método depende fortemente da escolha da função de onda de teste $\Psi_\text{trial}$. Uma função de teste mal escolhida pode levar a uma estimativa de energia pouco precisa.

2. **Aproximação por conjunto de bases**

O segundo método de aproximação surge na etapa de construção da função de onda — a abordagem por conjunto de bases. Em química quântica, resolver a equação de Schrödinger exatamente para moléculas é quase impossível. Em vez disso, aproximamos a função de onda complexa de muitos elétrons construindo-a a partir de funções matemáticas mais simples e previamente definidas. Um conjunto de bases é essencialmente uma coleção dessas funções matemáticas conhecidas, geralmente centradas nos átomos da molécula, que são usadas como blocos de construção para representar a forma e o comportamento dos elétrons no sistema. Pense nisso como tentar recriar uma escultura detalhada usando apenas um conjunto de peças padrão de LEGO — quanto maior a variedade e o número de peças disponíveis (isto é, quanto maior o conjunto de bases), mais fiel será a aproximação da forma original.

Essas funções de base são frequentemente inspiradas nas soluções analíticas de sistemas simples, como o átomo de hidrogênio, assumindo formas do tipo gaussiana ou do tipo Slater, embora ainda sejam aproximações. Em vez de trabalhar com orbitais moleculares teoricamente “exatos”, porém intratáveis, nós os expressamos como uma combinação linear (uma soma com coeficientes) dessas funções de base. Quando as funções de base se assemelham a orbitais atômicos, esse método é conhecido como a abordagem de Combinação Linear de Orbitais Atômicos (LCAO, do inglês *Linear Combination of Atomic Orbitals*). Ao otimizar os coeficientes dessa combinação linear, podemos encontrar a melhor função de onda e a melhor energia aproximadas possíveis dentro das limitações do conjunto de bases escolhido.

* Quanto maior o número de funções incluídas no conjunto de bases, melhor será a aproximação, porém ao custo de um maior esforço computacional.
* Um conjunto de bases pequeno fornece uma estimativa grosseira, enquanto um conjunto de bases maior produz resultados mais precisos, à custa de exigir mais recursos computacionais.

Em resumo, para tornar os cálculos viáveis e reduzir o custo computacional, utilizamos o princípio variacional ao aproximar a função de onda, o que reduz a complexidade computacional e permite uma otimização iterativa para minimizar a energia. Paralelamente, a abordagem por conjunto de bases simplifica os cálculos ao representar os orbitais atômicos como uma combinação de funções predefinidas, em vez de resolver diretamente uma função de onda contínua.

### VQE (Variational Quantum Eigensolver)

O **Variational Quantum Eigensolver (VQE)** é o principal método que utilizaremos para explorar o processo $H + H = H_2$ e, nesta seção, vamos analisar o que é o VQE e como ele funciona. Mas, antes disso, vamos fazer uma pausa para refletir sobre um ponto muito importante por meio da seguinte questão de verificação.

#### Verifique seu entendimento

<details>
  <summary>
    Se já existem tantas estratégias para resolver problemas de química, por que precisamos de um computador quântico? E qual é o propósito de usar computadores quânticos e clássicos em conjunto?
  </summary>


**Resposta:**

A computação quântica tem o potencial de revolucionar a química ao enfrentar problemas que os computadores clássicos têm dificuldade em resolver devido ao **crescimento exponencial** do espaço de estados quânticos. Richard Feynman observou de forma célebre que, para simular a natureza, as computações também precisam ser quânticas [ref 1].

Por exemplo, simular a molécula de **cafeína** usando o conjunto de bases mais simples (STO-3G) exigiria cerca de (10^{48}) bits, um número muito maior do que o total de estrelas no universo observável ((10^{24})) [ref 2]. Em contraste, um computador quântico pode descrever os orbitais eletrônicos da cafeína com cerca de **160 qubits**.

Os computadores quânticos processam naturalmente interações quânticas por meio de **superposição** e **emaranhamento**, oferecendo um caminho promissor para simulações moleculares precisas. Além disso, podemos combinar as vantagens de ambos os mundos:

* **computadores quânticos**, para a simulação explícita dos elétrons, e
* **computadores clássicos**, para o pré- e pós-processamento de dados, o gerenciamento do algoritmo, a otimização e outras tarefas auxiliares.

Espera-se que essa abordagem híbrida acelere a **descoberta de novos materiais**, o **desenvolvimento de fármacos** e a **previsão de reações químicas**, reduzindo experimentos caros baseados em tentativa e erro [ref 3][ref 4].

* *Emerging quantum computing algorithms for quantum chemistry*
  [https://arxiv.org/abs/2109.02873](https://arxiv.org/abs/2109.02873)
* *Chemistry Beyond Exact Solutions on a Quantum-Centric Supercomputer*
  [https://arxiv.org/html/2405.05068v1](https://arxiv.org/html/2405.05068v1)
* *Quantum-centric supercomputing for materials science: A perspective on challenges and future directions*
  [https://www.sciencedirect.com/science/article/pii/S0167739X24002012](https://www.sciencedirect.com/science/article/pii/S0167739X24002012)

</details>

Agora, vamos voltar ao **VQE**.

O VQE combina o poder de **computadores quânticos e clássicos**, utilizando fundamentalmente o **princípio variacional** para obter a energia do estado fundamental de um sistema. Para entender o VQE, é útil dividi-lo em três partes principais:

#### Observável (Quântico): o Hamiltoniano molecular (energia de uma molécula)

No VQE, o Hamiltoniano molecular/atômico é um **observável**, o que significa que podemos medir seu valor por meio de um experimento. Nosso objetivo é encontrar a **menor energia possível** (a energia do estado fundamental) da molécula. Para isso, utilizamos um **estado quântico de teste**, gerado por um circuito quântico parametrizado (ansatz). Medimos o observável e otimizamos o estado quântico até alcançar a menor energia possível.

O **conjunto de bases** utilizado para construir o Hamiltoniano molecular determina o número de qubits necessários e afeta diretamente a precisão do VQE. Escolher o conjunto de bases adequado é fundamental para equilibrar **eficiência** e **precisão**. Para simplificar os cálculos sem alterar o conjunto de bases, podemos empregar estratégias como **imposição de simetrias** e **redução do espaço ativo**. Muitas moléculas possuem formas simétricas (como uma borboleta ou um floco de neve), o que implica que certas partes se comportam da mesma maneira. Em vez de calcular tudo separadamente, podemos focar apenas nas partes únicas, economizando recursos quânticos ao explorar essas simetrias. Na redução do espaço ativo, consideramos apenas os orbitais mais relevantes, já que nem todos os elétrons impactam significativamente a energia molecular. Elétrons próximos ao núcleo tendem a permanecer praticamente inalterados, enquanto outros influenciam a ligação química. Ao aplicar esses métodos, conseguimos tornar o VQE mais eficiente mantendo a precisão.

Uma vez obtido o Hamiltoniano molecular com o conjunto de bases apropriado e as estratégias acima, precisamos **transformá-lo** em uma forma adequada para computadores quânticos. Mapear problemas para **operadores de Pauli** pode ser bastante complexo, especialmente em química quântica, que lida com partículas indistinguíveis (elétrons), enquanto qubits são distinguíveis. Não entraremos nos detalhes desses mapeamentos aqui, mas recomendamos os seguintes recursos: uma discussão geral sobre mapeamento de problemas para operadores quânticos pode ser encontrada em *Quantum computing in practice*; uma discussão mais detalhada sobre o mapeamento de problemas de química para operadores quânticos está em *Quantum chemistry with VQE*.

Neste módulo, forneceremos diretamente os Hamiltonianos apropriados (de **um qubit**) para (H) e (H_2), para que possamos focar no uso do computador quântico. Esses Hamiltonianos de um qubit são preparados usando o conjunto de bases **STO-6G** e o **mapeamento de Jordan–Wigner**, que é o mais direto e possui a interpretação física mais simples, pois mapeia a ocupação de um spin-orbital para a ocupação de um qubit. Também utilizamos uma **técnica de redução de qubits baseada em simetrias do Hamiltoniano**, que explora padrões no comportamento das ocupações de spin para reduzir o número de qubits. Para a molécula de (H_2), assumimos que a distância entre os dois átomos de hidrogênio é $0,735 \mathring{A}$.

---

#### Ansatz (Quântico): a função de onda de teste (como construir um estado quântico simples com um circuito quântico)

No VQE, o **ansatz** (plural: *ansätze*) consiste em dois componentes principais. O primeiro é a **preparação do estado inicial**, que define o estado do qubit aplicando portas quânticas **sem parâmetros variacionais**. O segundo componente é o **circuito quântico parametrizado**, um circuito especial com parâmetros ajustáveis — semelhantes a botões de um rádio. Esses parâmetros serão usados na etapa final, pelo **otimizador clássico**, para nos ajudar a alcançar o melhor estado fundamental possível.

Na seção sobre o princípio variacional, aprendemos que a qualidade do estado de teste influencia diretamente a qualidade dos resultados do algoritmo variacional. Isso significa que escolher um **bom ansatz** é crucial no VQE. Novamente, trata-se de um tema rico e complexo. Não abordaremos aqui os diferentes tipos de ansatz nem suas origens. Se você quiser se aprofundar em circuitos quânticos parametrizados e ansätze, pode explorar a lição *Ansatz and variational form* do curso *Variational algorithm design*, que oferece explicações detalhadas e exemplos.

Como utilizaremos um Hamiltoniano de **um qubit** neste módulo, precisamos de um **circuito quântico parametrizado de um qubit** como ansatz. Na próxima seção, veremos **três tipos de ansätze de um qubit**, comparando-os e discutindo os principais critérios para a escolha de um ansatz adequado.



#### (Clássico) Otimizador: ajuste fino do circuito quântico

Depois que o computador quântico mede a energia do observável a partir do ansatz, os **parâmetros do ansatz** e o **valor da energia** são enviados para um **otimizador clássico** para ajuste. Esse processo de otimização é realizado em um computador clássico, geralmente utilizando pacotes científicos de uso geral, como o **SciPy**.

O otimizador clássico trata a energia medida como uma **função de custo**. Em problemas de otimização, uma função de custo (também chamada de função objetivo) é uma função matemática que mede o quão “boa” é uma determinada solução. O objetivo do otimizador é encontrar o conjunto de parâmetros que **minimiza** essa função de custo. No contexto da busca pela energia do estado fundamental de uma molécula, a própria **energia** atua como a função de custo — queremos encontrar os parâmetros do circuito quântico (nossa “solução”) que resultem na menor energia possível.

O otimizador clássico utiliza esse valor de energia medido (o custo) para determinar o próximo conjunto de parâmetros otimizados para o ansatz quântico. Esses parâmetros atualizados são então enviados de volta ao circuito quântico, e o processo se repete. A cada iteração, o otimizador ajusta os parâmetros tentando reduzir a energia (minimizar a função de custo) até que um critério de convergência pré-definido seja atingido, idealmente garantindo que a menor energia possível (correspondente ao estado fundamental da molécula para aquela distância de ligação e conjunto de bases) seja encontrada.

Existem muitas estratégias de otimização disponíveis em pacotes científicos como o SciPy. Você pode conhecer mais no material **Optimization loops** do curso **Variational algorithm design**. Aqui, utilizaremos o **COBYLA** (*Constrained Optimization BY Linear Approximations*), um algoritmo de otimização adequado para paisagens de energia complexas. Em particular, o COBYLA **não tenta calcular gradientes** da função estudada; por isso, ele é classificado como um **otimizador livre de gradiente**.

Imagine que você está tentando encontrar o ponto mais alto de uma cadeia de montanhas de olhos vendados. Como não consegue ver toda a paisagem, você dá pequenos passos em diferentes direções, verificando se está subindo ou descendo. O COBYLA funciona de maneira semelhante — ele se move pelo espaço de parâmetros, testando diferentes valores e melhorando gradualmente o resultado até encontrar o melhor ponto.

## Calcular a energia do estado fundamental de um átomo de hidrogênio com VQE

Agora, vamos usar o que aprendemos para calcular a **energia do estado fundamental de um átomo de hidrogênio**.

Vamos começar carregando alguns **pacotes necessários**, incluindo as **Qiskit Runtime primitives**. Também iremos selecionar o **computador quântico menos ocupado** disponível para nós.

Há um trecho de código abaixo para **salvar suas credenciais no primeiro uso**. Certifique-se de **remover essas informações do notebook** depois de salvá-las no seu ambiente, para que suas credenciais não sejam compartilhadas acidentalmente ao compartilhar o notebook. Consulte os guias [Configurar sua conta IBM Cloud](/docs/guides/initialize-account) e [Inicializar o serviço em um ambiente não confiável](/docs/guides/cloud-setup-untrusted) para obter orientações adicionais.

In [ ]:
# Load the Qiskit Runtime service
from qiskit_ibm_runtime import QiskitRuntimeService

# Load the Runtime primitive and session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Syntax for first saving your token.  Delete these lines after saving your credentials.
# QiskitRuntimeService.save_account(channel='ibm_quantum_platform', instance = '<YOUR_IBM_INSTANCE_CRN>', token='<YOUR-API_KEY>', overwrite=True, set_as_default=True)
# service = QiskitRuntimeService(channel='ibm_quantum_platform')

# Load saved credentials
#service = QiskitRuntimeService()


from qiskit_aer import AerSimulator
backend = AerSimulator()

# Use the least busy backend, or uncomment the loading of a specific backend like "ibm_brisbane".
#backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
# backend = service.backend("ibm_brisbane")
print(backend.name)

### Passo 1: Mapear o problema para circuitos e operadores quânticos

Iniciamos o cálculo do VQE definindo o Hamiltoniano da molécula de hidrogênio ($H_2$) para uma distância de ligação específica. Esse Hamiltoniano representa a energia total do sistema em termos de operadores de qubits, tendo sido obtido e mapeado a partir do sistema molecular por meio de um procedimento padrão:

1. utilização do conjunto de bases STO-6G (uma coleção específica de funções matemáticas usada para aproximar os orbitais eletrônicos),
2. aplicação do mapeamento de Jordan–Wigner (uma técnica para traduzir operadores fermiônicos que descrevem elétrons em operadores de qubits), e
3. redução do número de qubits por meio do uso das simetrias do Hamiltoniano, a fim de simplificar o problema.

Como explicado anteriormente, as energias do estado fundamental calculadas dependem fortemente da escolha do conjunto de bases e da geometria molecular (como a distância de ligação). Para essa configuração específica e após todas essas transformações, o Hamiltoniano em termos de qubits resultante é simples:

$\hat{H} = -0.2355 I + 0.2355 Z$

Aqui, (I) representa o operador identidade e (Z) representa o operador de Pauli-Z, ambos atuando sobre um único qubit. Os coeficientes são derivados das integrais calculadas usando o conjunto de bases STO-6G nessa distância de ligação específica, após as transformações apropriadas.

Com esse Hamiltoniano definido, podemos agora utilizar o VQE para calcular sua energia do estado fundamental. É útil comparar a energia do estado fundamental obtida com valores esperados. Para um único átomo de hidrogênio isolado (H), a energia do estado fundamental é exatamente (-0{,}5) Hartree (na ausência de efeitos relativísticos). Vamos, então, calcular a energia exata do estado fundamental **do nosso Hamiltoniano de qubit específico**, conforme definido acima, e compará-la com valores conhecidos relevantes.

In [ ]:
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# Qubit Hamiltonian of the hydrogen atom generated by using STO-3G basis set and parity mapping
Hamiltonian = SparsePauliOp.from_list([("I", -0.2355), ("Z", 0.2355)])

# exact ground state energy of Hamiltonian

A = np.array(Hamiltonian)
eigenvalues, eigenvectors = np.linalg.eig(A)
print(
    "The exact ground state energy of the Hamiltonian is ",
    min(eigenvalues).real,
    "hartree",
)
h = min(eigenvalues.real)

Em seguida, precisamos de um circuito quântico parametrizado, chamado **ansatz**, para preparar uma função de onda de teste $\Psi_\text{teste}$ para o estado fundamental. O objetivo é encontrar os parâmetros $\theta$ que minimizam o valor esperado da energia
$
\langle \psi(\theta) | \hat{H} | \psi(\theta) \rangle.
$
A escolha do ansatz é crucial, pois ela determina o conjunto de estados quânticos que o circuito consegue preparar. Um ansatz “bom” é aquele que é flexível o suficiente para representar um estado muito próximo do verdadeiro estado fundamental do Hamiltoniano em estudo, mas não tão complexo a ponto de exigir muitos parâmetros ou circuitos muito profundos, o que seria inviável para os computadores quânticos atuais.

Aqui, vamos testar três ansätze diferentes de um único qubit para observar qual deles oferece uma melhor **“cobertura”** dos possíveis estados quânticos de um qubit. O termo “cobertura” refere-se ao conjunto de estados quânticos que o circuito ansatz consegue gerar ao variar seus parâmetros.

Utilizaremos três ansätze baseados em diferentes combinações de portas de rotação de um qubit:

* **Ansatz com rotação em um único eixo**:
  Esse ansatz utiliza rotações em torno de apenas um eixo $R_x(\theta)$. Na esfera de Bloch, isso corresponde a um movimento restrito a um círculo específico. Trata-se da opção menos flexível, cobrindo um conjunto limitado de estados.

* **Ansätze com rotações em dois eixos**:
  Esses ansätze combinam rotações em torno de dois eixos diferentes $(R_x(\theta_1) R_z(\theta_2)$ e $(R_x(\theta_1) R_z(\theta_2) R_x(\theta_3))$. Isso permite alcançar uma porção maior da esfera de Bloch em comparação com rotações em um único eixo.

Ao comparar os resultados do VQE obtidos com esses três ansätze, podemos analisar como a flexibilidade e a cobertura do espaço de estados influenciam nossa capacidade de encontrar a verdadeira energia do estado fundamental do Hamiltoniano simplificado. Um ansatz mais flexível tem o *potencial* de fornecer uma melhor aproximação, mas também pode tornar o processo de otimização clássica mais desafiador.


In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, DensityMatrix, Pauli

theta = Parameter("θ")
phi = Parameter("φ")
lam = Parameter("λ")

ansatz1 = QuantumCircuit(1)
ansatz1.rx(theta, 0)

ansatz2 = QuantumCircuit(1)
ansatz2.rx(theta, 0)
ansatz2.rz(phi, 0)

ansatz3 = QuantumCircuit(1)
ansatz3.rx(theta, 0)
ansatz3.rz(phi, 0)
ansatz3.rx(lam, 0)

Agora, vamos gerar **5000 números aleatórios para cada parâmetro** e plotar a distribuição dos estados quânticos aleatórios gerados pelos três ansätze usando esses parâmetros aleatórios. Você pode pensar nesses parâmetros como rotações em torno de diferentes eixos sobre uma superfície esférica.

Para visualizar a distribuição dos estados quânticos, utilizaremos a **Esfera de Bloch**, uma esfera tridimensional que representa o estado de um único qubit. Qualquer ponto na esfera corresponde a um possível estado do qubit, em que os polos norte e sul são análogos aos estados clássicos “0” e “1”, mas o qubit também pode ocupar qualquer ponto intermediário, exibindo propriedades quânticas especiais como a superposição.

Primeiro, prepare as funções necessárias para plotar a Esfera de Bloch em 3D e gere os 5000 parâmetros aleatórios.

In [ ]:
import matplotlib.pyplot as plt


def plot_bloch(bloch_vectors):
    # Extract X, Y, Z coordinates for 3D projection
    X_coords = bloch_vectors[:, 0]
    Z_coords = bloch_vectors[:, 2]

    # Compute Y coordinates from X and Z to approximate the full Bloch sphere projection
    Y_coords = bloch_vectors[:, 1]

    # Create 3D plot
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(X_coords, Y_coords, Z_coords, color="blue", alpha=0.6)

    # Labels and title
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("Parameterized 1-Qubit Circuit on 3D Bloch Sphere")

    # Set axis limits and make them equal
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])

    # Ensure equal aspect ratio for all axes
    ax.set_box_aspect([1, 1, 1])  # Equal scaling for x, y, z axes

    # Show grid
    ax.grid(True)

    plt.show()


num_samples = 5000  # Number of random states
theta_vals = np.random.uniform(0, 2 * np.pi, num_samples)
phi_vals = np.random.uniform(0, 2 * np.pi, num_samples)
lam_vals = np.random.uniform(0, 2 * np.pi, num_samples)

Vamos agora ver como funciona o nosso primeiro ansatz.

In [ ]:
# List to store Bloch Sphere XZ coordinates
bloch_vectors = []

# Generate quantum states and extract Bloch vectors
for i in range(num_samples):
    # Create a circuit and bind parameters
    qc = ansatz1
    bound_qc = qc.assign_parameters({theta: theta_vals[i]})  # , lam: lam_vals[i]})
    state = Statevector.from_instruction(bound_qc)
    rho = DensityMatrix(state)

    X = rho.expectation_value(Pauli("X")).real
    Y = rho.expectation_value(Pauli("Y")).real
    Z = rho.expectation_value(Pauli("Z")).real
    bloch_vectors.append([X, Y, Z])  # Store X, Z components

# Convert to a numpy array for plotting
bloch_vectors = np.array(bloch_vectors)

plot_bloch(bloch_vectors)

Podemos ver que o nosso **primeiro ansatz** gera um conjunto de estados quânticos distribuídos em forma de **anel** na Esfera de Bloch. Isso faz sentido, pois fornecemos ao ansatz apenas um único parâmetro de rotação. Dessa forma, ele só consegue produzir estados obtidos por rotações em torno de um único eixo. Partindo do ponto $(0,0,1)$ e realizando rotações em torno de um eixo fixo, o resultado será sempre um anel.

Vamos agora analisar o **segundo ansatz**, que possui duas portas de rotação ortogonais — `Rx` e `Rz`.

In [ ]:
bloch_vectors = []

# Generate quantum states and extract Bloch vectors
for i in range(num_samples):
    # Create circuit and bind parameters
    qc = ansatz2
    bound_qc = qc.assign_parameters(
        {theta: theta_vals[i], phi: phi_vals[i]}
    )  # , lam: lam_vals[i]})
    state = Statevector.from_instruction(bound_qc)
    rho = DensityMatrix(state)

    X = rho.expectation_value(Pauli("X")).real
    Y = rho.expectation_value(Pauli("Y")).real
    Z = rho.expectation_value(Pauli("Z")).real
    bloch_vectors.append([X, Y, Z])  # Store X, Z components

# Convert to numpy array for plotting
bloch_vectors = np.array(bloch_vectors)

plot_bloch(bloch_vectors)

Aqui, podemos observar que o **segundo ansatz** cobre uma porção maior da Esfera de Bloch — porém, note que os pontos ficam mais concentrados próximos aos polos e mais espalhados ao redor do equador. Agora é hora de analisar o **último ansatz**.

In [ ]:
bloch_vectors = []

# Generate quantum states and extract Bloch vectors
for i in range(num_samples):
    # Create circuit and bind parameters
    qc = ansatz3
    bound_qc = qc.assign_parameters(
        {theta: theta_vals[i], phi: phi_vals[i], lam: lam_vals[i]}
    )
    state = Statevector.from_instruction(bound_qc)
    rho = DensityMatrix(state)

    X = rho.expectation_value(Pauli("X")).real
    Y = rho.expectation_value(Pauli("Y")).real
    Z = rho.expectation_value(Pauli("Z")).real
    bloch_vectors.append([X, Y, Z])  # Store X, Z components

# Convert to numpy array for plotting
bloch_vectors = np.array(bloch_vectors)

plot_bloch(bloch_vectors)

Aqui, podemos ver estados quânticos distribuídos de forma **mais uniforme** gerados pelo nosso **último ansatz**.

Como mencionado, a melhor estratégia é adquirir algum conhecimento prévio sobre o estado fundamental que se deseja encontrar e escolher um ansatz que seja adequado para explorar estados próximos a esse estado fundamental. Por exemplo, se soubéssemos que o estado fundamental estivesse próximo de um dos polos, poderíamos escolher o **ansatz 2**. Por simplicidade, vamos manter o **ansatz 3**, que explora de maneira aproximadamente uniforme toda a Esfera de Bloch.

Agora que selecionamos o nosso ansatz, vamos desenhar o circuito.

In [ ]:
# Pre-defined ansatz circuit and operator class for Hamiltonian

ansatz = ansatz3

num_params = ansatz.num_parameters
print("This circuit has ", num_params, "parameters")

ansatz.draw("mpl", style="iqp")

### Passo 2: Otimizar para o hardware alvo

Ao executar um cálculo em um computador quântico real, não nos preocupamos apenas com a lógica abstrata do circuito quântico. Também precisamos considerar aspectos como quais operações podem ser realizadas por aquele computador quântico específico e onde, fisicamente, estão localizados os qubits que estamos utilizando. Eles estão próximos uns dos outros? Estão distantes?

Por isso, o próximo passo é reescrever o nosso circuito usando portas que sejam naturais para o hardware quântico que será utilizado, levando em conta também o *layout* dos qubits. Esse processo é conhecido como **transpilação** (*transpilation*). Após essa etapa, é possível ver o nosso ansatz simples convertido para um conjunto diferente de portas, e os qubits abstratos sendo mapeados para qubits físicos em um computador quântico real.


In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

config = backend.configuration()

#print("Backend: {config.backend_name}")
#print("Native gates: ", config.supported_instructions, ",")


target = backend.target

pm = generate_preset_pass_manager(target=target, optimization_level=3)

ansatz_isa = pm.run(ansatz)

ansatz_isa.draw(output="mpl", idle_wires=False, style="iqp")

Caso estivessemos executando em um computador quântico real, o circuito teria sido reescrito em termos das portas nativa do computador.

Também precisamos mapear o nosso Hamiltoniano de acordo com essas mudanças, como mostrado no código a seguir:


In [ ]:
Hamiltonian_isa = Hamiltonian.apply_layout(layout=ansatz_isa.layout)

### Passo 3: Executar no hardware alvo

Agora é o momento de executar o nosso VQE. Para isso, primeiro precisamos de uma **função de custo** para o processo de otimização, que avalia o valor esperado do Hamiltoniano para um estado quântico gerado pelo ansatz.

Não se preocupe! Você não precisa programar tudo do zero. Já preparamos uma função para isso, e tudo o que você precisa fazer é executar a célula abaixo.

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

Por fim, preparamos os **parâmetros iniciais** para o nosso ansatz e para o processo de otimização. Você pode simplesmente usar todos os valores iguais a zero ou valores aleatórios.

In [ ]:
# x0 = np.random.uniform(0, 2*pi, 3)
x0 = [1, 1, 0]

In [ ]:
# QPU Est. 2min for ibm_brisbane

from scipy.optimize import minimize
from qiskit_ibm_runtime import Batch

batch = Batch(backend=backend)

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
estimator = Estimator(mode=batch)
estimator.options.default_shots = 10000

res = minimize(
    cost_func,
    x0,
    args=(ansatz_isa, Hamiltonian_isa, estimator),
    method="cobyla",
    options={"maxiter": 10, "tol": 0.01},
)

batch.close()

In [ ]:
h_vqe = res.fun
print("The reference ground state energy is ", min(eigenvalues))
print("The computed ground state energy is ", h_vqe)

Parabéns! Você acabou de concluir com sucesso o seu **primeiro experimento de química quântica**. Podemos observar uma diferença entre a energia exata do estado fundamental do Hamiltoniano e o valor que obtivemos, mas, como utilizamos uma técnica padrão de mitigação de erros (que corrige erros de leitura), essa diferença é pequena. Isso é um excelente começo!

**Nota:** É possível obter um resultado ainda melhor ajustando o nível de mitigação de erros por meio do parâmetro [`resilience_level`](/docs/guides/configure-error-mitigation). O valor padrão é 1; ao definir um valor mais alto, será utilizado mais tempo de QPU, mas o resultado retornado pode ser mais preciso.

### Passo 4: Pós-processamento

Chegou a hora de analisar como o nosso **otimizador clássico** se comportou. Execute a célula abaixo e observe o **padrão de convergência**.

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0, 10, 10)

# Define the constant function
y_constant = np.full_like(x, h)
ax.plot(
    range(cost_history_dict["iters"]), cost_history_dict["cost_history"], label="VQE"
)
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost (Hartree)")
ax.plot(y_constant, label="Target")
plt.legend()
plt.draw()

Começamos com um valor inicial bastante bom, de modo que obtivemos um bom valor final em apenas **10 iterações**. É possível observar picos grandes e pequenos, o que é uma característica típica do otimizador **COBYLA** — ele explora o espaço de parâmetros como se não “enxergasse” a paisagem completa e ajusta o tamanho dos passos a cada avaliação.

#### Verifique seu entendimento

<details>
  <summary>
    Qual é a sua observação? Qual parte do processo acima pode ser aprimorada para obter resultados mais próximos dos valores teóricos ou da energia exata do estado fundamental do Hamiltoniano? Quais fatores devem ser considerados?
  </summary>

**Resposta:**

O primeiro ponto a ser considerado é a escolha do **conjunto de bases** utilizado no cálculo do Hamiltoniano molecular. Como mencionado anteriormente, a energia do estado fundamental do átomo de hidrogênio é $-0,5$ Hartree, um valor bem conhecido, e o conjunto de bases **STO-6G** escolhido não é suficientemente preciso para reproduzir esse valor com alta fidelidade.

A escolha de um conjunto de bases mais complexo aumenta o número de qubits necessários para representar o Hamiltoniano; consequentemente, torna-se necessário adotar um **ansatz mais elaborado e adequado** para problemas de química quântica.

Outro aspecto importante a ser otimizado é o **gerenciamento do ruído** presente na QPU. Técnicas mais avançadas de mitigação de erros tendem a produzir resultados melhores, embora geralmente demandem mais tempo de execução. Além disso, é fundamental considerar como o **número de *shots*** (`shot_number`) influencia a precisão dos resultados.

Por fim, um melhor desempenho de convergência também pode ser alcançado testando **diferentes otimizadores clássicos**, que podem se adaptar melhor à paisagem do problema em questão.

</details>

## Calcular a energia do estado fundamental da molécula de hidrogênio com VQE

Agora que analisamos o processo geral do VQE usando átomos de hidrogênio ($H$), vamos calcular de forma mais direta a **energia do estado fundamental da molécula de hidrogênio ($H_2$)**.

### Passo 1: Mapear o problema para circuitos e operadores quânticos

Aqui também fornecemos um Hamiltoniano de **um qubit** que utiliza o conjunto de bases **STO-6G** e a transformação de **Jordan–Wigner**, com redução do número de qubits por meio do uso de uma simetria do Hamiltoniano. Observe que foi utilizada uma distância internuclear entre os dois átomos de hidrogênio de **(0{,}735\ \mathring{A})**.

Diferentemente do cálculo para um único átomo de hidrogênio ((H)), para calcular o estado fundamental da molécula de hidrogênio ((H_2)) precisamos considerar também a **força repulsiva entre os núcleos** dos dois átomos, além da energia associada aos orbitais eletrônicos. Nesta etapa, forneceremos esse termo como uma **constante**, e iremos de fato calcular esse valor no exercício de verificação (*check-in problem*). O Hamiltoniano efetivo é dado por: $\hat{H} = -1.04886 I + -0.79674 Z + 0.18122 X$

In [ ]:
h2_hamiltonian = SparsePauliOp.from_list(
    [("I", -1.04886087), ("Z", -0.7967368), ("X", 0.18121804)]
)

# exact ground state energy of hamiltonian
nuclear_repulsion = 0.71997
A = np.array(h2_hamiltonian)
eigenvalues, eigenvectors = np.linalg.eig(A)
print("Electronic ground state energy (Hartree): ", min(eigenvalues).real)
print("Nuclear repulsion energy (Hartree): ", nuclear_repulsion)
print(
    "Total ground state energy (Hartree): ", min(eigenvalues).real + nuclear_repulsion
)
h2 = min(eigenvalues).real + nuclear_repulsion

### Passo 2: Otimizar para o hardware alvo

Como o número de qubits utilizado pelo VQE e pelo Hamiltoniano anteriores é o mesmo do *backend* que será usado na execução, iremos reutilizar o **ansatz existente** e a sua **forma já otimizada**.

In [ ]:
h2_hamiltonian_isa = h2_hamiltonian.apply_layout(layout=ansatz_isa.layout)

### Passo 3: Executar no hardware alvo

Agora é hora de realizar os cálculos em uma **QPU real**. Quase tudo permanece igual ao caso anterior, mas desta vez utilizaremos um **ponto inicial apropriado** para se ajustar melhor ao Hamiltoniano considerado. Além disso, durante a parte iterativa do processo, algumas configurações do **`Estimator`**, que é utilizado para calcular os valores esperados do Hamiltoniano para o ansatz na QPU, serão definidas de forma ligeiramente diferente em relação aos cálculos anteriores.

Discutiremos essa mudança com mais detalhes em uma **questão de verificação (*check-in question*)**.

In [ ]:
x0 = [2, 0, 0]

In [ ]:
# QPU time 4min for ibm_brisbane
batch = Batch(backend=backend)

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
estimator = Estimator(mode=batch)
estimator.options.default_shots = 10000

res = minimize(
    cost_func,
    x0,
    args=(ansatz_isa, h2_hamiltonian_isa, estimator),
    method="cobyla",
    options={"maxiter": 15},
)

batch.close()

In [ ]:
h2_vqe = res.fun + nuclear_repulsion
print(
    "The reference ground state energy is ", min(eigenvalues).real + nuclear_repulsion
)
print("The computed ground state energy is ", h2_vqe)

Apesar de o VQE fornecer teoricamente um **limite superior** para a verdadeira energia do estado fundamental, implementações práticas em hardware quântico real ou em simulações ruidosas, bem como as aproximações feitas na construção do Hamiltoniano (como a escolha do conjunto de bases ou a redução do número de qubits), podem introduzir erros que, em alguns casos, resultam em uma energia medida ligeiramente **menor** do que o valor teórico exato ou do que uma referência numérica específica.

Ainda assim, apesar da presença desses erros, os resultados parecem **satisfatórios**, especialmente considerando o **pequeno número de iterações** utilizadas. Agora, vamos finalizar este cálculo de VQE analisando como o **otimizador** se comportou.

### Passo 4: Pós-processamento


In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0, 5, 15)

# Define the constant function
y_constant = np.full_like(x, min(eigenvalues))
ax.plot(
    range(cost_history_dict["iters"]), cost_history_dict["cost_history"], label="VQE"
)
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost (Hartree)")
ax.plot(y_constant, label="Target")
plt.legend()
plt.draw()

#### Verifique seu entendimento

Vamos calcular a **energia de repulsão nuclear** da molécula de (H_2), que incluímos como um valor constante ((0{,}71997) Hartree).

![Molécula de H2](/learning/images/modules/computer-science/vqe/h2.avif)

<details>
  <summary>
    Por favor, utilize a <a href="https://en.wikipedia.org/wiki/Coulomb%27s_law">lei de Coulomb</a> e o sistema de <a href="https://en.wikipedia.org/wiki/Atomic_units">unidades atômicas</a> para garantir que o valor final esteja em Hartree.
  </summary>

**Resposta:**

Como ambos os núcleos de hidrogênio são carregados positivamente, eles se repelem devido à força eletrostática. Essa repulsão é descrita pela lei de Coulomb:

$E_{\text{repulsiva}} = \frac{e^2}{4\pi\epsilon_0 R},$

onde (e) é a carga do próton, (\epsilon_0) é a permissividade do vácuo e (R) é a distância entre os dois núcleos, medida em metros ou em raios de Bohr, resultando em energia em joules (J).

Para calcular essa energia em Hartree, precisamos converter a expressão acima para o sistema de **Unidades Atômicas (AU)**. Nesse sistema, temos (e^2 = 1), (4\pi\epsilon_0 = 1) e o raio de Bohr ((a_0)) é igual a 1, tornando-se a escala fundamental de comprimento. Com essas simplificações, a lei de Coulomb se reduz a:

$E_{\text{repulsão}} = \frac{1}{R},$

onde (R) deve ser medido em raios de Bohr ((a_0)).

Para converter a separação nuclear fornecida em (\mathring{A}) para (a_0), usamos a relação:

$1,\mathring{A} = 1{,}88973,a_0$

Assim, (0{,}735,\mathring{A}) se torna:

$0{,}735 \times 1{,}88973 = 1{,}38895,a_0.$

Portanto, a energia de repulsão nuclear da molécula de (H_2) considerada é:

$E_{\text{repulsão}} = \frac{1}{R} = \frac{1}{1{,}38895} = 0{,}71997\ \text{Hartree}.$

</details>


## Calcular a energia de reação de (H + H = H_2)

Agora vamos usar o que já obtivemos! Você utilizou o VQE (*Variational Quantum Eigensolver*) para calcular a energia do estado fundamental do átomo de hidrogênio ((H)) e da molécula de hidrogênio ((H_2)). O que falta agora é usar esses valores calculados para obter a **energia de reação** do processo (H + H = H_2).

A **energia de reação** é a variação de energia que ocorre quando substâncias reagem para formar novas substâncias. Pense nisso como construir algo: às vezes é necessário fornecer energia (como empilhar blocos) e, em outras situações, a energia é liberada (como uma bola descendo uma ladeira). Em química, as reações podem **absorver energia** (*endotérmicas*) ou **liberar energia** (*exotérmicas*).

A energia de reação do processo (H + H = H_2) pode ser calculada pela seguinte fórmula:

$E_{reaction} = E_{H_2} - (E_H + E_H)$


Ao executar a célula abaixo, vamos visualizar esse resultado. Aqui, utilizaremos o **valor exato da energia do estado fundamental** de cada Hamiltoniano e compararemos a energia de reação obtida pela **solução exata** com aquela obtida pelos **resultados do VQE**.

In [ ]:
# Theoretical values
E_H_theo = h.real
E_H2_theo = h2

# Experimental values
E_H_exp = h_vqe
E_H2_exp = h2_vqe

# Calculate reaction energies
E_reaction_theo = E_H2_theo - (2 * E_H_theo)
E_reaction_exp = E_H2_exp - (2 * E_H_exp)

# Set up the plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xlim(0, 3)
ax.set_ylim(-1.16, -0.93)  # Adjust y-axis range to highlight differences
ax.set_xticks([])
ax.set_ylabel("Energy (Hartree)")
ax.set_title("H + H → H₂ Reaction Energy Diagram")

# Plot theoretical energy levels
ax.hlines(
    y=2 * E_H_theo, xmin=0.5, xmax=1.3, linewidth=2, color="r", label="2H (Exact)"
)
ax.hlines(y=E_H2_theo, xmin=1.3, xmax=2, linewidth=2, color="b", label="H₂ (Exact)")

# Plot experimental energy levels
ax.hlines(
    y=2 * E_H_exp,
    xmin=0.5,
    xmax=1.5,
    linewidth=2,
    color="r",
    linestyle="dashed",
    label="2H (VQE)",
)
ax.hlines(
    y=E_H2_exp,
    xmin=1.5,
    xmax=2.5,
    linewidth=2,
    color="b",
    linestyle="dashed",
    label="H₂ (VQE)",
)

# Add labels
ax.text(
    1,
    2 * E_H_theo,
    f"2H: {2*E_H_theo:.4f}",
    verticalalignment="top",
    horizontalalignment="left",
)
ax.text(
    2,
    E_H2_theo,
    f"H₂: {E_H2_theo:.4f}",
    verticalalignment="top",
    horizontalalignment="left",
)
ax.text(
    1,
    2 * E_H_exp,
    f"2H_VQE: {2*E_H_exp:.4f}",
    verticalalignment="bottom",
    horizontalalignment="right",
)
ax.text(
    2,
    E_H2_exp,
    f"H₂_VQE: {E_H2_exp:.4f}",
    verticalalignment="bottom",
    horizontalalignment="right",
)

# Add arrows for reaction energy with ΔE label in the middle
mid_y_theo = (2 * E_H_theo + E_H2_theo) / 2
mid_y_exp = (2 * E_H_exp + E_H2_exp) / 2
ax.annotate(
    "",
    xy=(1.3, E_H2_theo),
    xytext=(1.3, 2 * E_H_theo),
    arrowprops=dict(arrowstyle="<->", color="g"),
)
ax.text(
    1.35, mid_y_theo, f"ΔE: {E_reaction_theo:.4f}", color="g", verticalalignment="top"
)

ax.annotate(
    "",
    xy=(1.5, E_H2_exp),
    xytext=(1.5, 2 * E_H_exp),
    arrowprops=dict(arrowstyle="<->", color="g", linestyle="dashed"),
)
ax.text(
    1.55,
    mid_y_exp,
    f"ΔE_VQE: {E_reaction_exp:.4f}",
    color="g",
    verticalalignment="center",
)

# Add legend
ax.legend()

plt.show()

Como mostrado na figura, embora existam alguns erros, a energia exata do estado fundamental dos Hamiltonianos e a energia de reação calculada a partir dos resultados do VQE são semelhantes, ficando próximas de **−0,2 Hartree**.

Vale notar que a energia de reação desse processo possui **valor negativo**, o que significa que **energia é liberada** durante a reação, e a molécula resultante possui **energia menor** do que a de dois átomos isolados.

### 6. Conclusão

Vamos resumir o que aprendemos até aqui.

Primeiro, analisamos duas técnicas de aproximação fundamentais para resolver problemas de química quântica: o **princípio variacional** e a **escolha do conjunto de bases**, ambas essenciais para o VQE. Exploramos o princípio variacional de forma analítica, calculando manualmente a energia do estado fundamental do oscilador harmônico simples.

Em seguida, estudamos o **VQE**, um algoritmo amplamente utilizado para calcular a energia do estado fundamental de um sistema quântico. Executamos códigos para calcular as energias do estado fundamental do átomo de hidrogênio ((H)) e da molécula de hidrogênio ((H_2)). Em particular, aprendemos que é necessário obter o **Hamiltoniano molecular apropriado** para o sistema e transformá-lo em uma forma que possa ser executada em um computador quântico. Também vimos que o **ansatz**, um circuito quântico parametrizado, é necessário para preparar estados quânticos de teste dentro do VQE, e discutimos a importância de escolher uma estrutura de ansatz adequada. Além disso, aprendemos que o VQE depende de um **processo iterativo de otimização clássica**, no qual um computador clássico guia o circuito quântico na busca pelo estado de menor energia, e observamos como esse processo converge.

Por fim, utilizamos as **energias do estado fundamental** de (H) e (H_2), obtidas por meio do VQE, para calcular a **energia de reação** do processo

$H + H \rightarrow H_2.$

O **VQE** é um algoritmo quântico poderoso para a era *near-term*, mas é importante estar ciente de suas limitações. O desempenho do VQE depende fortemente da **escolha do ansatz** — encontrar um ansatz que seja ao mesmo tempo eficiente de preparar e capaz de representar com precisão o verdadeiro estado fundamental torna-se um desafio para moléculas maiores e mais complexas. Além disso, o hardware quântico atual é suscetível a **ruído**, o que pode afetar a precisão dos resultados do VQE, especialmente para circuitos mais profundos ou com um maior número de qubits.

Apesar desses desafios, o VQE serve como um **algoritmo fundamental**, e pesquisas em andamento estão explorando métodos variacionais mais sofisticados e **técnicas de mitigação de erros** para expandir os limites do que é possível fazer em química quântica com computadores quânticos de curto prazo. Por exemplo, algoritmos como a **Diagonalização Quântica Baseada em Amostragem** (*Sample-based Quantum Diagonalization – SQD*) vêm sendo desenvolvidos, combinando amostras obtidas a partir de circuitos quânticos com diagonalização clássica em um subespaço. Essa abordagem busca melhorar a estimativa de energia e contornar algumas das limitações do VQE, especialmente no que diz respeito à **eficiência de medições** e à **robustez frente ao ruído**.


## Revisão e perguntas

### Conceitos-chave

* Um **algoritmo quântico variacional** é um paradigma de computação no qual um computador clássico e um computador quântico trabalham juntos para resolver um problema.
* No **VQE**, começamos com o Hamiltoniano do sistema e o mapeamos em qubits para execução no computador quântico. Selecionamos um circuito quântico parametrizado, chamado **ansatz**, e realizamos medições repetidas, variando os parâmetros do ansatz até que o menor valor de energia seja alcançado. A busca no espaço de parâmetros é feita por um **otimizador clássico**. Para obter bons resultados, é necessário escolher um bom ansatz e um otimizador apropriado.
* A **energia de reação** é a variação total de energia em uma reação química, determinada pela diferença entre a energia dos reagentes e a dos produtos.

---

### Verdadeiro / Falso

1. **Verdadeiro** — O princípio variacional afirma que o valor esperado da energia para qualquer função de onda de teste é sempre maior ou igual à verdadeira energia do estado fundamental.
2. **Verdadeiro** — Um conjunto de bases é uma coleção de funções usadas para aproximar funções de onda quânticas.
3. **Falso** — O VQE não resolve exatamente a equação de Schrödinger; ele fornece uma aproximação variacional.
4. **Verdadeiro** — No VQE, um circuito quântico parametrizado (ansatz) é usado para preparar funções de onda de teste.
5. **Falso** — A escolha do otimizador impacta sim a convergência e a qualidade do resultado.
6. **Verdadeiro** — O `Estimator` do Qiskit é usado para calcular diretamente os valores esperados dos Hamiltonianos no VQE.

---

### Questões de múltipla escolha

1. **Qual é o propósito do Hamiltoniano no VQE?**
   **Resposta:** **B)** Determinar a energia dos estados quânticos

2. **Qual é o principal objetivo do algoritmo VQE?**
   **Resposta:** **A)** Encontrar a energia do estado fundamental de um Hamiltoniano

3. **Quantos estados quânticos foram gerados neste notebook para comparar os ansätze?**
   **Resposta:** **C)** 5000

4. **Por que um otimizador clássico é necessário no VQE?**
   **Resposta:** **B)** Atualizar os parâmetros do ansatz para minimizar a energia

5. **Por que o ansatz é projetado para ser parametrizado?**
   **Resposta:** **B)** Para permitir a exploração de um amplo espaço de estados quânticos

6. **Qual das alternativas é a mais correta sobre a escolha de um bom ansatz?**
   **Resposta:** **B)** Um ansatz deve ser adaptado ao sistema para garantir que ele consiga gerar estados próximos ao estado fundamental

## (Opcional) Apêndice: Sobrecarga do otimizador em função da complexidade do ansatz

O VQE enfrenta diversos desafios bem conhecidos [ref. 6], e os pontos a seguir estão diretamente relacionados ao que aprendemos até aqui.

### 1. Desafios na escolha do ansatz

Existe um desafio inerente na seleção do ansatz variacional adequado. Ansätze inspirados na química (como o **UCCSD**) oferecem maior fidelidade física, mas exigem circuitos profundos. Por outro lado, ansätze **hardware-efficient** possuem circuitos mais rasos, porém podem carecer de interpretabilidade física. Além disso, muitos ansätze introduzem um número excessivo de parâmetros variacionais que pouco contribuem para a melhoria da precisão, mas aumentam significativamente a dificuldade do processo de otimização.

### 2. Dificuldades de otimização

A paisagem de otimização do VQE pode conter regiões onde os gradientes se anulam exponencialmente (os chamados **barren plateaus**), dificultando que os otimizadores clássicos atualizem os parâmetros variacionais de forma eficiente. Para lidar com isso, pesquisadores têm explorado diferentes tipos de otimizadores — baseados em gradiente e livres de gradiente —, mas ambos apresentam desafios. Otimizadores baseados em gradiente sofrem com os barren plateaus, enquanto métodos livres de gradiente geralmente exigem um grande número de avaliações da função de custo.

### 3. Sobrecarga do otimizador

Outro desafio importante é a **sobrecarga do otimizador**, que está relacionada à escala do problema. À medida que o tamanho do problema aumenta, os circuitos quânticos necessários para o VQE tornam-se mais profundos e complexos; isso normalmente também aumenta o número de parâmetros a serem otimizados. Como consequência, o processo de otimização pode se tornar intratável, levando a uma convergência lenta e a dificuldades em encontrar a solução ótima.

Aqui, vamos analisar esses desafios utilizando o VQE para a molécula de (H_2), empregando **dois tipos diferentes de ansätze**.

*(Nota: isso pode consumir mais tempo de QPU; portanto, sinta-se à vontade para utilizar um simulador caso não tenha tempo suficiente.)*

In [ ]:
from qiskit.circuit import ParameterVector

num_iter = 4
alpha = ParameterVector("alpha", 3)
beta = ParameterVector("beta", 3 * num_iter)

# step1: Map problem to quantum circuits and operators
hamiltonian = SparsePauliOp.from_list(
    [("I", -1.04886087), ("Z", -0.7967368), ("X", 0.18121804)]
)

ansatz_1 = ansatz3
ansatz_2 = QuantumCircuit(1)
for i in range(num_iter):
    ansatz_2.rx(beta[i * 3 + 0], 0)
    ansatz_2.rz(beta[i * 3 + 1], 0)
    ansatz_2.rx(beta[i * 3 + 2], 0)

In [ ]:
ansatz_1.draw("mpl")

In [ ]:
ansatz_2.draw("mpl")

In [ ]:
# Step 2: Optimize for target hardware

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

ansatz_isa_1 = pm.run(ansatz_1)
ansatz_isa_2 = pm.run(ansatz_2)
hamiltonian_isa_1 = hamiltonian.apply_layout(layout=ansatz_isa_1.layout)
hamiltonian_isa_2 = hamiltonian.apply_layout(layout=ansatz_isa_2.layout)

Now let's run a VQE with an initial point made of all ones, with a maximum of 20 steps, and compare the convergence of both runs.



In [ ]:
# QPU time 3m 40s for ibm_brisbane
# Step 3: Execute on target hardware

from scipy.optimize import minimize

x0 = np.ones(ansatz_1.num_parameters)

batch = Batch(backend=backend)


cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
estimator = Estimator(mode=batch)
estimator.options.default_shots = 2048

res = minimize(
    cost_func,
    x0,
    args=(ansatz_isa_1, hamiltonian_isa_1, estimator),
    method="cobyla",
    options={"maxiter": 20},
)

batch.close()

In [ ]:
# Save Cost_history as a new list
ansatz_1_history = cost_history_dict["cost_history"]

In [ ]:
# QPU time 3m 40s for ibm_brisbane

x0 = np.ones(ansatz_2.num_parameters)

batch = Batch(backend=backend)


cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
estimator = Estimator(mode=batch)
estimator.options.default_shots = 2048

res = minimize(
    cost_func,
    x0,
    args=(ansatz_isa_2, hamiltonian_isa_2, estimator),
    method="cobyla",
    options={"maxiter": 20},
)

batch.close()

In [ ]:
ansatz_2_history = cost_history_dict["cost_history"]

In [ ]:
fig, ax = plt.subplots()

# Define the constant function)
ax.plot(
    range(cost_history_dict["iters"]),
    ansatz_1_history,
    label="Ansatz with 3 parameters",
)
ax.plot(
    range(cost_history_dict["iters"]),
    ansatz_2_history,
    label="Ansatz with 12 parameters",
)
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost (Hartree)")
plt.legend()
plt.draw()

O gráfico acima demonstra claramente que o processo de otimização do **ansatz com mais variáveis** leva mais tempo para alcançar uma **convergência estável**.

Em vez de depender de circuitos simples de um único qubit e de um ansatz direto, a complexidade da otimização aumenta quando são necessários **circuitos quânticos maiores** e **ansätze estruturados mais complexos**. Isso evidencia um desafio bem conhecido do VQE: a **sobrecarga do otimizador**.

Pesquisadores continuam desenvolvendo diversas metodologias avançadas para utilizar computadores quânticos em problemas de química. Você pode acessar uma variedade de materiais educacionais no **IBM Quantum Learning** em [/learning](/learning).

## Referências

*   \[[ref 1](https://link.springer.com/article/10.1007/BF02650179) ] Richard P. Feynman, Simulating Physics with Computers, International Journal of Theoretical Physics, 1982.
*   [\[ref 2\]](https://link.springer.com/chapter/10.1007/978-1-4614-8730-2_10) Marov, M.Y. (2015). The Structure of the Universe. In: The Fundamentals of Modern Astrophysics. Springer, New York, NY.
*   \[[ref 3](https://www.ibm.com/quantum/blog/photoresists-quantum-chemistry-jsr)] How to solve difficult chemical engineering problems with quantum computing, IBM Research Blog, 2023.
*   \[[ref 4](https://ieeexplore.ieee.org/document/8585034)] Y. Cao, J. Romero and A. Aspuru-Guzik, "Potential of quantum computing for drug discovery," in IBM Journal of Research and Development, vol. 62, no. 6, pp. 6:1-6:20, 1 Nov.-Dec. 2018
*   \[[ref 5](https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.32.170)] Present State of Molecular Structure Calculation, REv. Mod. Phys. 32, 170, 1960
*   \[[ref 6](https://jmsh.springeropen.com/articles/10.1186/s41313-021-00032-6)] Fedorov, D.A., Peng, B., Govind, N. et al. VQE method: a short survey and recent developments. Mater Theory 6, 2 (2022)



© IBM Corp., 2017-2026